In [39]:
import glob, sys
import numpy as np
import matplotlib.pyplot as plt
import time
import pyccl as ccl
cosmo = ccl.Cosmology(Omega_c = 0.30711 - 0.048254, Omega_b = 0.048254, h = 0.677, sigma8 = 0.8288, n_s=0.96)

In [40]:
%load_ext autoreload
%autoreload 2
import CL_COUNT_class_cluster_abundance as cl_count

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


initialize the `ClusterAbundance` object

In [41]:
clc = cl_count.ClusterAbundance()
#choose the halo mass function and mass definition
massdef = ccl.halos.massdef.MassDef('vir', 'critical', c_m_relation=None)
hmd = ccl.halos.hmfunc.MassFuncDespali16(cosmo, mass_def=massdef)
clc.set_cosmology(cosmo = cosmo, hmd = hmd, massdef = massdef)
clc.sky_area = (0.25)*4*np.pi
clc.f_sky = clc.sky_area/4*np.pi

# Binned approach
Compute the 2D integral

## model

$$
N_{\alpha\beta}^{\rm predicted} = \Omega_s\int_{z_\alpha}^{z_{\alpha + 1}} dz\int_{\log_{10}M_\beta}^{\log_{10}M_{\beta + 1}}\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}d\log_{10}M
$$

define mass & redshift binning

In [42]:
z_corner = np.linspace(0.25, 1.25, 10)
logm_corner = np.linspace(14, 14.8, 10)
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]
Z_bin = binning(z_corner)
logMass_bin = binning(logm_corner)

In [43]:
#define redshift-mass grid
z_grid = np.linspace(0., 3, 300)
logm_grid = np.linspace(12,16, 300)

`simps`

### Abundance prediction

In [44]:
t0 = time.time()
clc.compute_multiplicity_grid_MZ(z_grid = z_grid, logm_grid = logm_grid)
Abundance_simps = clc.Cluster_Abundance_MZ(Redshift_bin = Z_bin, Proxy_bin = logMass_bin, method = 'simps')
t1 = time.time()
print('time (seconds) = ' + str(t1-t0))
time_simps = t1-t0

time (seconds) = 0.27968406677246094


### Shot noise

In [45]:
shot_noise_variance = np.diag(Abundance_simps.flatten())

In [47]:
covariance = shot_noise_variance #here, we do not take account of sample variance

## data

In [48]:
N_obs = np.random.multivariate_normal(Abundance_simps.flatten(), covariance)

## Likelihood

In [49]:
%load_ext autoreload
%autoreload 2
import CL_COUNT_class_likelihood as lnL

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
Likelihood = lnL.Likelihood()

### Binned Gaussian

$L = \mathcal{G}(\vec{\widehat{N}}|\vec{\mu}, \Sigma_{SV})$

In [51]:
lnL_Binned_Gaussian = Likelihood.lnLikelihood_Binned_Gaussian(Abundance_simps, N_obs.reshape(len(Z_bin), len(logMass_bin)), covariance)
print(Likelihood.lnL_Binned_Gaussian)

-35.8961460624269


### Binned Poisson

$L = \prod_i^N\mathcal{P}(\widehat{N}_i| \mu_i)$

In [52]:
lnL_Binned_Poissonian = Likelihood.lnLikelihood_Binned_Poissonian(Abundance_simps, N_obs.reshape(len(Z_bin), len(logMass_bin)))
print(Likelihood.lnL_Binned_Poissonian)

1770808.2766462842


# Unbinned Approach

## data

In [54]:
#Generate mass, redshift catalog
n_cluster = 100000
redshift, M200c = np.random.random(n_cluster)*(1 - 0.2) + 0.2, 10**(np.random.random(n_cluster)*(15 - 13) + 13)

## model

$$
\lambda(m_i, z_i) = \Omega_s\frac{d^2V(z_i)}{dz d\Omega}\frac{dn(M_i,z_i)}{d\log_{10}M}
$$

In [55]:
t0 = time.time()
clc.compute_multiplicity_grid_MZ(z_grid = z_grid, logm_grid = logm_grid)
multiplicity_interp = clc.multiplicity_function_individual_MZ(z = redshift, logm = np.log10(M200c), method = 'interp')
t1 = time.time()
print('time (seconds) = ' + str(t1-t0))

time (seconds) = 0.17561888694763184


In [56]:
N_tot = clc.Cluster_Abundance_MZ(Redshift_bin = [[.2, 1]], Proxy_bin = [[13,15]], method = 'simps')[0][0]

## Likelihood

$\log L = \sum_i^N \log\lambda(m_i, z_i) - N_{tot}$

In [57]:
Likelihood.lnLikelihood_UnBinned_Poissonian(multiplicity_interp, N_tot)
print(Likelihood.lnL_UnBinned_Poissonian)

-4698765.870197004
